<center><h1>EARTH 468 Project: Randomness in the global earthquake distribution</h1></center>

Submitted by: Prithvi Thakur  
Email: prith@umich.edu

In [1]:
# import modules
from plotly import __version__, tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

import numpy as np
import random
import scipy.stats as stats
from haversine import haversine
import plotly.graph_objs as go
import pandas as pd
from IPython.core.display import display, HTML

In [ ]:
# Toggle code button to hide code
HTML('''
<script>
    var code_show=true; //true -> hide code at first

    function code_toggle() {
        $('div.prompt').hide(); // always hide prompt

        if (code_show){
            $('div.input').hide();
        } else {
            $('div.input').show();
        }
        code_show = !code_show
    }
    $( document ).ready(code_toggle);
</script>
<a href="javascript:code_toggle()">[Toggle Code]</a>
''')

<h3>Load USGS global earthquake catalogue</h3>

In [2]:
# global earthquake data obtained from USGS with magnitude cutoff 7.0
# (https://earthquake.usgs.gov/earthquakes)
data = pd.read_csv("data.csv")

lat = np.array(data["latitude"])
lon = np.array(data["longitude"])
y = np.array(data["time"])
n = len(y)  # number of data points

# convert time yyyy-mm-dd-h-m-s to continuous format
year = np.zeros(n)
mm = np.zeros(n)
dd = np.zeros(n)
h = np.zeros(n)
m = np.zeros(n)
s = np.zeros(n)

for i in range(len(y)):
    year[i] = float(y[i][0:4])
    mm[i] = float(y[i][5:7])
    dd[i] = float(y[i][8:10])
    h[i] = float(y[i][11:13])
    m[i] = float(y[i][14:16])
    s[i] = float(y[i][17:23])

# time = from 0 to 117 (1900-2017) in years
time = (year-1900) + (mm + (dd + (h + (m + s/60)/60)/24)/30)/12 

depth = np.array(data["depth"])
depth_error = np.array(data["depthError"])

Mw = np.array(data["mag"])
Mw_error = np.array(data["magError"])

<b>Caveat:</b> I have assumed each month to contain 30 days, and ignored the leap year effects when considering the time of earthquakes.

<h3>Plot</h3>

In [4]:
trace = go.Scatter(x = year, y = Mw, mode='markers',
                   marker = dict(color = 'white', 
                                 line = dict(width=1, color = 'red')))

data1 = [trace]
layout = go.Layout(title='Earthquake Magnitude (Mw) vs Year',
                   xaxis= dict(title='Year'),
                   yaxis =dict(title='Moment Magnitude (Mw)'),
                   showlegend = False)

fig = go.Figure(data=data1, layout=layout)
iplot(fig)

<h3>Declustering</h3>

We remove events fo which the preceding larger earthquakes occur within 3 years and 1000km range temporaly and spatially. 

<b>Declustering Algorithm</b>

* Obtain the window (time: 3 years, distance: 1000km) for each event
* For n data points, we have n windows
* Get the biggest event for each window
* We have n biggest events.
* Merge the same events.
* Repeat the above five steps until every event is non repeating.

In [5]:
def window(t, lat, lon, time, Mw):
    """This function takes an input argument of time of one earthquake (t) 
        and returns the indices of earthquakes that happened within 3 years 
        (post given earthquake) and 1000 km of that earthquake."""
    
    # index of the input earthquake
    idx = np.where(time == t)[0] 
    
    # time window: given time + 3 yrs
    idx1 = np.where(time <= t + 3)[0]
    idx2 = np.where(time >= t)[0]
    time_idx = np.sort(np.array(list(set(idx1) & set(idx2))))
    
    # Position of the given event
    x1 = lat[idx]
    y1 = lon[idx]
    
    # Position of the other events in that time window
    x2 = lat[time_idx]
    y2 = lon[time_idx]
    space_idx = np.zeros(len(time_idx)).astype(int)
    
    # Space window: get index where distance is less than 1000 km
    for i in range(len(time_idx)):
        if haversine([x1, y1], [x2[i], y2[i]]) < 1000:
            space_idx[i] = time_idx[i]
    
    return list(filter(lambda x: x!=0, space_idx))

In [6]:
def decluster(N, lat, lon, time, Mw):
    """This function computes an array of largest 
        event in each window"""
    
    # index of the largest event in each window
    largest_event = np.zeros(N).astype(int)

    for i in range(N):
        win = window(time[i], lat, lon, time, Mw)
        if len(win) <= 1:
            # when there is no clustering
            # i.e. only one event in the window
            largest_event[i] = i
            continue
        # index where magnitude is maximum
        max_idx = np.where(Mw[win] == np.max(Mw[win]))[0][0]
        largest_event[i] = win[max_idx]
        
    return np.unique(largest_event)

In [7]:
# maximum number of iterations for declustering
iter_max = 10;

# local variables for functions
N = n;
lat_d = lat;
lon_d = lon;
time_d = time;
Mw_d = Mw;

# run the algorithm multiple times until the number of events saturate
print("The total number of events in the declsutered catalogue after each iteration")
print(N)
for i in range(iter_max):
    largest_event = decluster(N, lat_d, lon_d, time_d, Mw_d)
    Mw_d = Mw_d[largest_event]
    lat_d = lat_d[largest_event]
    lon_d = lon_d[largest_event]
    time_d = time_d[largest_event]
    N = len(largest_event)
    print(len(largest_event))

The total number of events in the declsutered catalogue after each iteration
1353
844
757
742
738
737
737
737
737
737
737


<b>Question:</b> If within a window for declustering, you have more than one earthquake with exactly the same magnitude, how would you choose the mainshock? Does this happen often? My code unconditionally chooses the first one as the mainshock.

<h3> Plot declustered data

In [8]:
trace3 = go.Scatter(x = np.round(time_d)+1900, y = Mw_d, mode='markers',
                   marker = dict(color = 'white', 
                                 line = dict(width=1, color = 'red')))

data3 = [trace3]
layout = go.Layout(title='Declustered Earthquake Magnitude (Mw) vs Year',
                   xaxis= dict(title='Year'),
                   yaxis =dict(title='Moment Magnitude (Mw)'),
                   showlegend = False)

fig = go.Figure(data=data3, layout=layout)
iplot(fig)

In [9]:
print("The number of events in the declustered catalogue with Mw>=8.5: ",
      sum(Mw_d[i] >= 8.5 for i in range(N)))

The number of events in the declustered catalogue with Mw>=8.5:  16


The above number is coherent with Shearer and Stark (2012). Note that there are no earthquakes with Mw>=8.5 after 2012 in the declustered catalogue.

<h3> Plot annual rates of $M\geq8, M\geq7.5, M\geq7$ earthquakes

In [10]:
# round off the months, days, hours
year_d = np.round(time_d)+1900
unique_year_d = np.unique(year_d)
n_year = len(unique_year_d)
#print(year_d)
# Yearly rate of earthquakes
rate_M70 = np.zeros(n_year)
rate_M75 = np.zeros(n_year)
rate_M80 = np.zeros(n_year)

for i in range(n_year):
    annual_idx =  np.where(year_d == unique_year_d[i])[0]
    rate_M70[i] = sum(Mw_d[annual_idx] >= 7)
    rate_M75[i] = sum(Mw_d[annual_idx] >= 7.5)
    rate_M80[i] = sum(Mw_d[annual_idx] >= 8)

In [11]:
trace4 = go.Scatter(x = unique_year_d, y = rate_M80, mode='lines', name='Mw>8.0')
trace5 = go.Scatter(x = unique_year_d, y = rate_M75, mode='lines', name='Mw>7.5')
trace6 = go.Scatter(x = unique_year_d, y = rate_M70, mode='lines', name='Mw>7.0')

layout = go.Layout(title='Annual rates of earthquakes',
                   xaxis= dict(title='Year'),
                   yaxis =dict(title='Rate (number/year)'))

fig = tools.make_subplots(rows=3, cols=1)

fig.append_trace(trace4, 1, 1)
fig.append_trace(trace5, 2, 1)
fig.append_trace(trace6, 3, 1)

fig['layout'].update(title='Annual rates of earthquakes (number/year)',
                   xaxis= dict(title='Year'))

iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]



<h3> Monte carlo simulations

We generate random independent, identically distributed (iid) earthquakes within the same range as observed. The time array after declustering is represented in number of days from 239.29 to 42,469.94. The number of events in this range in the declustered catalogue is 737. 

We randomly generate exactly the same number of earthquakes as observed in the declustered catalogue within the given time period [0 to 42,470], and estimate the probability of elevated rate of large earthquake activity. We simulate 100,000 catalogues in this manner, under the null hypothesis assumption that the earthquake distribution follows a Poisson process in time.

    Estimated Probability = fraction of the 100,000 catalogues that have atleast the observed number of events within an interval of specified length. 

In [14]:
# seed the random number with system time
random.seed()

# time in days
days = time_d*12*30

# number of events
n_events = len(time_d)

# simulated array of earthquake magnitudes in the domain
sim_Mw = np.zeros(n_events)

# simulated array of time in days
sim_days = np.zeros(n_events)

#for i in range(n_sim):
def mc_simulation():
    sim_days = [random.uniform(0, np.max(days)) for x in range(n_events)]
    
    # Gutenberg-Richter distribution
    gr_b = 1.3 # Shearer and Stark
    freq = lambda M: n_events*(10**(-gr_b*M))
    
    it = 0;
    while it<n_events:
        mag2 = random.uniform(7, 9.51)
        prob = freq(mag2)/freq(7)
        if prob > random.random():
            sim_Mw[it] = mag2
            it = it+1
    return sim_days, sim_Mw

<b> Statistics from observations

In [15]:
# Number of earthquakes with magnitude cutoffs 8, 8.5
n_80 = sum(Mw_d>=8)
n_85 = sum(Mw_d>=8.5)

# time of the above earthquakes
time_80 = [days[i] for i in range(N) if Mw_d[i]>=8]
time_85 = [days[i] for i in range(N) if Mw_d[i]>=8.5]

# magnitude of the earthquakes
m_80 = [Mw_d[i] for i in range(N) if Mw_d[i]>=8]
m_85 = [Mw_d[i] for i in range(N) if Mw_d[i]>=8.5]

# plot the earthquakes with magnitude cutoff 8, 8.5
trace7 = go.Scatter(x = time_80, y = m_80, mode='markers', name='Mw>8.0')
trace8 = go.Scatter(x = time_85, y = m_85, mode='markers', name='Mw>8.5')

fig = tools.make_subplots(rows=2, cols=1)

fig.append_trace(trace8, 1, 1)
fig.append_trace(trace7, 2, 1)

fig['layout'].update(title='Observed large earthquakes with magnitude cutoffs 8.0 and 8.5',
                   xaxis= dict(title='No. of days'))
iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]



Looking at the first graph above, there are 5 events within the time period of 37,826 days to 40,451 days (counting from 1900). Due to the approximations used in converting time, I can not pinpoint the exact earthquakes and their dates, but the 37,826th day earthquake corresponds to the 2004 Sumatra-Andaman earthquake, and 40,451 corresponds to the 2012 Sumatra earthquake (Duputel et al, 2012). Thus, there are 5 events out of the 16, with $M_w \geq 8.5$, that occur within 2626 days of each other.

Looking at the recent seismicity elevation in the second graph between 36,346 days and 42,399 days, we have 22 out of 87 earthquakes occuring within this time period of 6053 days.

In [16]:
# number of simulations
n_sim = 10000

# Counter for number of times there are more than given 
# events within specified interval
counter80 = 0
counter85 = 0

# loop to repeat monte carlo simulations n times
for i in range(n_sim):
    # simulated array of earthquake magnitudes in the domain
    sim_Mw = np.zeros(n_events)

    # simulated array of time in days
    sim_days = np.zeros(n_events)
    
    # monte carlo simulation
    [sim_days, sim_Mw] = mc_simulation()
    
    # Filter earthquakes with magnitude cutoffs
    # Number of earthquakes with magnitude cutoffs 8, 8.5
    n2_80 = sum(sim_Mw>=8)
    n2_85 = sum(sim_Mw>=8.5)

    # time of the above earthquakes
    time2_80 = [sim_days[i] for i in range(N) if sim_Mw[i]>=8]
    time2_85 = [sim_days[i] for i in range(N) if sim_Mw[i]>=8.5]

    # magnitude of the earthquakes
    m2_80 = [sim_Mw[i] for i in range(N) if sim_Mw[i]>=8]
    m2_85 = [sim_Mw[i] for i in range(N) if sim_Mw[i]>=8.5]
    
    # if there are more than 5 Mw>=8.5 EQs within 2626 days interval
    for j in range(n2_85):
        if j >= n2_85-5 or n2_85 <5:
            break;
        if abs(time2_85[j+5] - time2_85[j]) <= 2626:
            #print(abs(time2_85[j+5] - time2_85[j]))
            counter85 = counter85 + 1;
            break;
            
    # if there are more than 22 Mw>=8.0 EQs within 6053 days interval
    for j in range(n2_80):
        if j >= n2_80-22 or n2_80 <22:
            break;
        if abs(time2_80[j+22] - time2_80[j]) <= 6053:
            counter80 = counter80 + 1;
            break;

In [17]:
counter85
counter80

9545

I simulated earthquakes following Poisson process 10,000 times. Out of the 10,000 simulations of earthquakes, 2912 simulations had more than 5 events of magnitude greater than 8.5 occuring within 2626 days of each other. This implies there is 29.12 percent chance that under the null hypothesis, we can have seismicity distribution equivalent to the observed seismicity clustering.

Also, there were 9508 out of 10,000 simulations with more than 22 earthquakes of magnitude greater than or equal to 8 within a 6053 days interval. Thus, there is a 95% chance that the observations follow a null hypothesis for Poisson process.

<h3> Kolmogorov-Smirnov (KS) test

In [31]:
# Every measurement is the measurement of time in this section

# filter the observation with magnitude cutoff
M70 = [days[i] for i in range(N) if Mw_d[i] >= 7]
M75 = [days[i] for i in range(N) if Mw_d[i] >= 7.5]
M80 = [days[i] for i in range(N) if Mw_d[i] >= 8]

bins = np.arange(0,np.max(days),20)

H1_70,_ = np.histogram(M70, bins = bins, normed=True)
H1_75,_ = np.histogram(M70, bins = bins, normed=True)
H1_80,_ = np.histogram(M70, bins = bins, normed=True)

# KS statistic
ks70 = np.zeros(n_sim)
ks75 = np.zeros(n_sim)
ks80 = np.zeros(n_sim)

# P-value
p70 = 0
p75 = 0
p80 = 0

n_sim = 1000

# loop to repeat monte carlo simulations n times
for i in range(n_sim):
    # simulated array of earthquake magnitudes in the domain
    sim_Mw = np.zeros(n_events)

    # simulated array of time in days
    sim_days = np.zeros(n_events)
    
    # monte carlo simulation
    [sim_days, sim_Mw] = mc_simulation()
    
    # Filter simulations with magnitude cutoffs
    sim70 = sim_days
    sim75 = [sim_days[i] for i in range(N) if sim_Mw[i]>=7.5]
    sim80 = [sim_days[i] for i in range(N) if sim_Mw[i]>=8]
    
    # calculate KS statistic for each simulation wrt observations
    H2_70,_ = np.histogram(sim70, bins = bins, normed=True)
    H2_75,_ = np.histogram(sim75, bins = bins, normed=True)
    H2_80,_ = np.histogram(sim80, bins = bins, normed=True)
    
    D70 = np.abs(H2_70 - H1_70)
    D75 = np.abs(H2_75 - H1_75)
    D80 = np.abs(H2_80 - H1_80)
    
    ks70[i] = D70[np.argmax(D70)]
    ks75[i] = D75[np.argmax(D75)]
    ks80[i] = D80[np.argmax(D80)]

In [40]:
#stats.kstest(H2_80, H1_80)
ks80
#DD = np.abs(H2_80 - stats.uniform(0, np.max(H2_80)).cdf)
trace10 = go.Histogram(x = ks70, name='Mw>7.0')
trace11 = go.Histogram(x = ks75, name='Mw>7.5')
trace12 = go.Histogram(x = ks80, name='Mw>8.0')

fig = tools.make_subplots(rows=1, cols=1)

#fig.append_trace(trace10, 1, 1)
fig.append_trace(trace12, 1, 1)
#fig.append_trace(trace12, 1, 3)

fig['layout'].update(title='Histogram of the KS test statistics with magnitude cutoff 8.0',
                   xaxis= dict(title='KS test statistic values values'), 
                     yaxis= dict(title='Frequency'))

#data = [go.Histogram(x = ks80)]
#fig = go.Figure(data=data)
iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]



In [28]:
np.max(days)

42469.946866087957

<h3> Poisson's dispersion test

<h3> Multinomial chi-squared test

In [144]:
# 1. divide the time into 100 intervals.
k = 100
interval = np.round(np.max(days))/k

# number of events per interval
N_k = np.zeros(k)

for i in range(k):
    N_k[i] = sum(days[j] < (interval*(i+1)) and \
                 days[j] > (interval*i) for j in range(N))
    
N_k = N_k.astype(int)

# 2. estimate the theoretical rate of events per interval
lamda = N/k

# 3. 

In [145]:
E_c = k*np.exp(-lamda)*

array([ 3,  2,  3,  1,  2,  8,  2,  4,  4,  7,  4,  3,  5,  7,  4,  5,  7,
        7,  7,  7,  6,  3, 10,  8, 10,  4,  9,  8,  9,  5,  6,  9, 11, 14,
        6,  8, 12, 11,  5,  8,  8,  4,  6,  9,  9,  9, 10,  5, 11,  9,  4,
        7,  4,  5, 10,  8,  8,  5, 12, 10, 10,  9,  8,  8, 10,  7,  9,  4,
        5,  5,  8,  8,  7,  9,  5,  6,  9,  8,  9,  5,  6, 12,  6,  5,  7,
       11, 11,  7,  9,  8,  7, 12,  7,  9,  9, 13, 10,  8, 15,  9])

In [146]:
lamda

7.37